In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

58892288/58889256 [==============================] - 0s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f1e81b69898> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e81b6def0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e81b1b8d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1e812dd8d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e812dd780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e812ed7f0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1e812f7208> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e812f7b00> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e812fee10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f1e7007cdd8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f1e70081400> Fals

In [0]:
# load data into dataframe with 'path' and 'target' columns
import pandas as pd
train_label_df = pd.read_csv('/data.csv', delimiter=' ', header=1, names=["uid"])
train_label_df["path"] = train_label_df['uid'].apply(lambda x : '/data/' + x.split(',')[-1][1:].split('/')[-1])
train_label_df["target"] = train_label_df['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[2]))
train_label_df =train_label_df.drop(['uid'], axis=1)

train_label_df2 = pd.read_csv('/data_temp.csv', delimiter=' ', header=1, names=["uid"])
train_label_df2["path"] = train_label_df2['uid'].apply(lambda x : "/data/" + x.split(',')[0] + ".jpg")
train_label_df2["target"] = train_label_df2['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[3]))
train_label_df2 =train_label_df2.drop(['uid'], axis=1)

In [0]:
train_df = pd.concat([train_label_df, train_label_df2], axis=0).reset_index().drop(['index'], axis=1)

In [0]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('/data/') if isfile(join('/data/', f))]

In [6]:
new_df = train_df
j = 0
for i in train_df['path']:
    if i.split("/")[-1] not in onlyfiles:
        new_df = new_df.drop(train_df.index[train_df['path'] == i].tolist())
    else:
      j+=1
train_df = new_df.reset_index().drop(['index'], axis=1)
train_df

,path,target
0,/content/drive/My Drive/team7/data/B9dZzI1n-EV...,0.112025
1,/content/drive/My Drive/team7/data/B9cQMfVnkMe...,0.638448
2,/content/drive/My Drive/team7/data/B9a0IQWnY9M...,0.053981
3,/content/drive/My Drive/team7/data/B9NnTUrHoNX...,0.143885
4,/content/drive/My Drive/team7/data/B9XuP83gx_j...,0.030143
...,...,...
2030,/content/drive/My Drive/team7/data/B_UgmnQj0tQ...,0.041773
2031,/content/drive/My Drive/team7/data/B_QoOOxl-ve...,0.024106
2032,/content/drive/My Drive/team7/data/B_OM7wVoCUd...,0.025871
2033,/content/drive/My Drive/team7/data/B_QTlktFkRZ...,0.014322


In [0]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(train_df, test_size=0.2)

In [8]:
batch_size = 128

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        x_col = 'path',
        y_col = 'target',
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='other',
) 

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test,
        x_col = 'path',
        y_col = 'target',
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='other',
)

Found 1628 validated image filenames.
Found 407 validated image filenames.


In [10]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


Train model pruned model

In [9]:
! pip install tensorflow-model-optimization

     |████████████████████████████████| 174kB 2.7MB/s 
     |████████████████████████████████| 296kB 8.4MB/s 


In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity
import tensorflow as tf
import numpy as np

In [18]:
import tempfile
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

Writing training logs to /tmp/tmpk1li84lb


In [0]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
p_model = load_model('/content/drive/My Drive/model42604.h5',)

In [13]:
p_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 150, 150, 3)]     0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              33558528  
_________________________________________________________________
fc2 (Dense)                  (None, 1024)              4195328   
_________________________________________________________________
pred (Dense)                 (None, 1)                 1025      
Total params: 52,469,569
Trainable params: 44,834,305
Non-trainable params: 7,635,264
_________________________________________

In [14]:
epochs = 15
end_step = np.ceil(1.0 * 1628 / 128).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.20,
                                                   final_sparsity=0.80,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = tf.keras.Sequential()
for layer in p_model.layers:
  if "fc" in layer.name:
    new_pruned_model.add(sparsity.prune_low_magnitude(layer, **new_pruning_params))
  else :
    new_pruned_model.add(layer)
new_pruned_model.summary()
new_pruned_model.compile(
    loss="mse",
    optimizer='adam',
    metrics=['mse'])

195
Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
prune_low_magnitude_fc1 (Pru (None, 4096)              67112962  
_________________________________________________________________
prune_low_magnitude_fc2 (Pru (None, 1024)              8389634   
_________________________________________________________________
pred (Dense)                 (None, 1)                 1025      
Total params: 90,218,309
Trainable params: 44,834,305
Non-trainable params: 45,384,004
_________________________________________________________________


In [0]:
new_pruned_model.fit(
        train_generator,
        steps_per_epoch=1280,
        epochs=15,
        callbacks=callbacks,
        validation_data=test_generator,
        validation_steps=1280,
        verbose = 2,
        )
final_pruned_model_test = sparsity.strip_pruning(pruned_model)
final_pruned_model_test.summary()
final_pruned_model_test.save("/content/drive/My Drive/model_pruned_final2.h5",include_optimizer=False)

Epoch 1/15
1280/1280 - 4486s - loss: 0.3839 - mse: 0.3816 - val_loss: 0.0302 - val_mse: 0.0301
Epoch 2/15
1280/1280 - 3317s - loss: 0.3790 - mse: 0.3776 - val_loss: 0.0306 - val_mse: 0.0301
Epoch 3/15
1280/1280 - 3231s - loss: 0.3775 - mse: 0.3791 - val_loss: 0.0297 - val_mse: 0.0302
Epoch 4/15
1280/1280 - 3202s - loss: 0.3855 - mse: 0.3806 - val_loss: 0.0306 - val_mse: 0.0301
Epoch 5/15
1280/1280 - 3194s - loss: 0.3812 - mse: 0.3786 - val_loss: 0.0299 - val_mse: 0.0302
Epoch 6/15
1280/1280 - 3232s - loss: 0.3803 - mse: 0.3811 - val_loss: 0.0308 - val_mse: 0.0301
Epoch 7/15
1280/1280 - 3176s - loss: 0.3814 - mse: 0.3786 - val_loss: 0.0300 - val_mse: 0.0302
Epoch 8/15
1280/1280 - 3235s - loss: 0.3785 - mse: 0.3816 - val_loss: 0.0301 - val_mse: 0.0302
Epoch 9/15
1280/1280 - 3239s - loss: 0.3807 - mse: 0.3782 - val_loss: 0.0295 - val_mse: 0.0302
Epoch 10/15
1280/1280 - 3246s - loss: 0.3835 - mse: 0.3816 - val_loss: 0.0298 - val_mse: 0.0302
Epoch 11/15


In [45]:
final_pruned_model_test = sparsity.strip_pruning(pruned_model)
final_pruned_model_test.summary()
final_pruned_model_test.save("/content/drive/My Drive/model_pruned_final.h5",include_optimizer=False)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 150, 150, 3)]     0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              33558528  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
_________________________________________________________________
dropout (Dropout)            (None, 1)                 0   